## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Humidity,Cloudiness,Wind Speed
0,0,Delta Del Tigre,UY,60.91,light rain,-34.7633,-56.3853,79,100,8.14
1,1,Ushuaia,AR,42.46,scattered clouds,-54.8000,-68.3000,75,40,20.71
2,2,Viedma,AR,52.84,broken clouds,-40.8135,-62.9967,76,74,11.01
3,3,Mitsamiouli,KM,80.80,scattered clouds,-11.3847,43.2844,77,32,9.75
4,4,Sal Rei,CV,70.25,scattered clouds,16.1833,-22.9167,84,30,13.31


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Humidity,Cloudiness,Wind Speed
3,3,Mitsamiouli,KM,80.80,scattered clouds,-11.3847,43.2844,77,32,9.75
7,7,Georgetown,MY,82.33,few clouds,5.4112,100.3354,94,20,2.30
8,8,Aliaga,PH,85.50,clear sky,15.5167,120.8667,59,1,3.71
10,10,Cayenne,GF,82.09,few clouds,4.9333,-52.3333,1,20,9.22
12,12,Dingle,PH,82.58,overcast clouds,10.9995,122.6711,75,100,8.63
26,26,Kavaratti,IN,85.24,overcast clouds,10.5669,72.6420,73,100,20.96
33,33,Bonthe,SL,76.64,moderate rain,7.5264,-12.5050,92,66,3.24
40,40,Marrakesh,MA,80.67,clear sky,31.6315,-8.0083,22,0,4.61
42,42,Pasni,PK,82.65,clear sky,25.2631,63.4710,71,0,9.24
46,46,Victoria,HK,79.18,overcast clouds,22.2855,114.1577,90,100,10.00


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull()

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Humidity,Cloudiness,Wind Speed
3,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False
12,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
1837,False,False,False,False,False,False,False,False,False,False
1840,False,False,False,False,False,False,False,False,False,False
1841,False,False,False,False,False,False,False,False,False,False
1843,False,False,False,False,False,False,False,False,False,False


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Mitsamiouli,KM,80.80,scattered clouds,-11.3847,43.2844,
7,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
8,Aliaga,PH,85.50,clear sky,15.5167,120.8667,
10,Cayenne,GF,82.09,few clouds,4.9333,-52.3333,
12,Dingle,PH,82.58,overcast clouds,10.9995,122.6711,
26,Kavaratti,IN,85.24,overcast clouds,10.5669,72.6420,
33,Bonthe,SL,76.64,moderate rain,7.5264,-12.5050,
40,Marrakesh,MA,80.67,clear sky,31.6315,-8.0083,
42,Pasni,PK,82.65,clear sky,25.2631,63.4710,
46,Victoria,HK,79.18,overcast clouds,22.2855,114.1577,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file,index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Description} at {Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))